# Deploying Puppet to Clients

## 1. Puppet Nodes

When managing fleets of computers, we usually want some rules to apply to every computer, and other rules to apply only to a subset of systems. Let's say you're managing all your servers with Puppet. You might want to install a basic set of tools on all of them, but only install the packages for serving web pages in your web servers. And only install the packages for sending and receiving email in your mail servers. There's a bunch of different ways that we can do this. In an earlier video, we saw how to conditionally apply some rules using facts from the machines. Another way to apply different rules to different systems is to use separate node definitions. In Puppet terminology, a **node** *is any system where we can run a Puppet agent.* It could be a physical workstation, a server, a virtual machine, or even a network router, as long as it has a Puppet agent and can apply the given rules. So we can set up Puppet to give some basic rules to all the nodes, but then apply some specific rules to the nodes that we want to be different. Let's check out an example of how this could look. When setting up Puppet, we usually have a default node definition that lists the classes that should be included for all the nodes. For example, it could look something like this.

![img11](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img11.jpg?raw=true)

Here, the default node is including two classes, the sudo class and the ntp class. For the ntp class, we're setting an additional servers parameter that lists the servers we can use to get the network time.

As you can see here, when defining a node, you can include a class by just using its name if there's no additional settings, or include the class and set additional parameters if necessary. 

All right, that's the default node, so it will apply to computers in the fleet by default. What if you want some settings to only apply to some specific nodes? 

![img12](https://github.com/Brian-E-Nguyen/Google-IT-Automation-with-Python/blob/5-Config-Management-and-Cloud/5-Config-Management-and-Cloud/Week-2-Deploying-Puppet/img/img12.jpg?raw=true)

You can do that by adding more node definitions that list the classes that you want them to include, like this. We can see here that specific nodes in the fleet are identified by their FQDNs, or fully qualified domain names. In this case, we have the node definition for a host called webserver.example.com. For this node, we're including the same sudo and ntp classes as before, and we're adding the apache class on top. We're listing the same classes because the classes included in the default node definition are only applied to the nodes that don't have an explicit entry. In other words, when a node requests which rules it should apply, Puppet will look at the node definitions, figure out which one matches the node's FQDN, and then give only those rules. To avoid repeating the inclusion of all the common classes, we might define a base class that does the work of including all the classes that are common to all node types. 

Now, where's this information stored? The node definitions are typically stored in a file called site.pp, which isn't part of any module. Instead, it just defines what classes will be included for what nodes. This is another step towards helping us organize our code in a way that makes it easier to maintain. Up next, we'll look into the infrastructure used by Puppet to verify if a node really has the name that it claims to have.